# Image augmentaion:

this notebook helps your to augment your annotations.
    1. Rotate (0 - 360)
    2. Add sharpness (random)
    3. Add brightness (random)

In [ ]:
from time import sleep
from utils import *

# Directory in which images exists 
DIR_PATH = "/media/sohaib/additional_/DataScience/Upwork_orders/people_walk_in_zebra_crossing/Mask---RCNN-Polygons-/data/zebra_crossing/"

# Json File Path
doc_labels = "/media/sohaib/additional_/DataScience/Upwork_orders/people_walk_in_zebra_crossing/Mask---RCNN-Polygons-/data/zebra_crossing/annotation2.json"

# Parsing Json
images = parse_via_json(doc_labels, DIR_PATH)

# Getting all of the classes from your annotations 
classes = list(set([ __ for _ in images.values() for __ in list(_.keys())]))
print(classes)

# Images you are going to augment should be greater than 1
if len(images) <= 1:
    print("I need at least 2 images")
    exit()

# Spliting train and testing images
total_training = int(len(images) * 0.9)
counter = 0

# Initilzing Train and val multiprocessing augmentation class
train = Augmentation(DIR_PATH + "/train/")
val = Augmentation(DIR_PATH + "/val/")


# Creating images
print("Creating Images....")
for image_path in images:
    if  counter < total_training:

        train.run(image_path, images[image_path])
    else:
        val.run(image_path, images[image_path])
    counter += 1


while train.pending > 0 or val.pending > 0:
    print(f"Train Pending  :{train.pending},  Validation Pending: {val.pending}", end= '\r')
    
    sleep(1)





print("Saving Labels into json")
save_mrcnn_labels(train.augmented_images, DIR_PATH + "/train/via_region_data.json", classes)
save_mrcnn_labels(val.augmented_images, DIR_PATH + "/val/via_region_data.json", classes)
file1 = open(DIR_PATH+"labels.txt","w")
write='"' + '","'.join(classes) + '"'
file1.writelines(write)
file1.close()


Traceback (most recent call last):
  File "/media/sohaib/additional_/DataScience/Upwork_orders/people_walk_in_zebra_crossing/Mask---RCNN-Polygons-/2_image_augmentation/utils.py", line 168, in parse_via_json
    region_name = list(region['region_attributes']['class'].keys())[0]
IndexError: list index out of range


['zebra_crossing']
Creating Images....


# Validation of Augmentation

Validating Images are augmentated right or not

In [ ]:


import random

# Path of augmented train path
base_path = f'{doc_labels.rsplit("/",1)[0]}/train/'
json_file_path = f"{base_path}/via_region_data.json"
labels = json.load(open(json_file_path))

# Getting all of the images
images = glob.glob(base_path + "*.*g")

# Getting the random sample from the images
image_list = random.sample(images,10)


# Getting x and y and ploting the images
for image_name in image_list:
    object_name = image_name.rsplit("/",1)[1].rsplit(".",1)[0]
    img = Image.open(f'{image_name}')

    points = []
    for i in range(len(labels[f'{object_name}.json']['regions'])):
        x = labels[f'{object_name}.json']['regions'][i]['shape_attributes']['all_points_x']
        y = labels[f'{object_name}.json']['regions'][i]['shape_attributes']['all_points_y']

        points.append(list(zip(x,y)))
    
    mask = make_mask(points,img.size)
    
    display([img , mask])

